# 01. Data Combination
Combine data from all sources into one **huge** dataframe

In [4]:
# change wd to repo home
import os
if os.getcwd().split('\\')[-1] != 'FM skills prediction':
    os.chdir('..')

In [6]:
import pandas as pd

In [11]:
# load FBRef data in separate dfs
fbref_dfs = {filename : pd.read_csv(f'data/FBRef_split/{filename}', index_col=0)
             for filename in os.listdir('data/FBRef_split')}

In [12]:
# combine FBRef dfs by type first
table_types = ['defense', 'gca', 'keepersadv', 'keepers', 'misc', 'passingtypes',
               'passing', 'playingtime', 'possession', 'shooting', 'stats']

fbref_dfs_types = {}

for t in table_types:
    fbref_dfs_types[t] = {}
    for k in fbref_dfs.keys():
        if k.split('_')[1] == t:
            fbref_dfs_types[t][k.split('_')[0]] = fbref_dfs[k]
    fbref_dfs_types[t] = pd.concat(fbref_dfs_types[t], axis=0, ignore_index=True).set_index(['_Player', '_Squad', '_Born'])

In [13]:
# combine FBRef dfs into one
fbref_combined_df = pd.concat(fbref_dfs_types, axis=1).reset_index()
fbref_combined_df.columns = fbref_combined_df.columns.map(lambda x: '_'.join(map(str, x)))

In [14]:
# store a complete FBRef file
fbref_combined_df.to_csv('data/FBRef_combined.csv')

This dataframe still contains multiple duplicates and a crazy number of columns. Duplicates are there becuse some players played for more than one team in a season. Columns are so numerous because I've been lazy and did not bother cleaning them

To deal with player duplicates, we will first remove players with fewer than 900 minutes (less that 10 '90s')

In [28]:
fbref_combined_filtered_df = fbref_combined_df[fbref_combined_df['misc__90s']>=10]
fbref_combined_filtered_df.drop_duplicates(['_Player_'], inplace=True)

C:\Users\alibe\AppData\Local\Temp\ipykernel_41292\1161894005.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  fbref_combined_filtered_df.drop_duplicates(['_Player_'], inplace=True)


In [29]:
# load FM data
fm_fd = pd.read_csv('data/FM 2023.csv')

We will then combine using an 'inner' join, so only a single lecord per player is kept. If a player played more than 10 90s for more than one team, only one of the records would be kept, but this will do for now

In [32]:
combined_data = fbref_combined_filtered_df.merge(right=fm_fd, left_on='_Player_', right_on='Name', how='inner')

In [33]:
combined_data.shape

(1380, 384)

In [34]:
combined_data.head()

,_Player_,_Squad_,_Born_,defense__Rk,defense__Nation,defense__Pos,defense__Age,defense__90s,defense_Tackles_Tkl,defense_Tackles_TklW,...,World reputation,Race,RCA,Colour of skin,Date of birth,Number of national team appearances,Goals scored for the national team,Salary,Rental club,UID
0,Karim Adeyemi,Dortmund,2002,3.0,de GER,FWMF,20.0,15.4,16.0,11.0,...,5750,African_Caribbean,125,12,2002/1/18,4,1,115067.0,NaN,16279486
1,Amine Adli,Leverkusen,2000,4.0,ma MAR,FWMF,22.0,15.9,18.0,10.0,...,4250,African_Caribbean,120,13,2000/5/10,0,0,23014.0,NaN,49037698
2,Naouirou Ahamada,Stuttgart,2002,6.0,fr FRA,MF,20.0,15.5,17.0,11.0,...,4096,African_Caribbean,112,16,2002/3/29,0,0,14383.0,NaN,49048373
3,Ludovic Ajorque,Mainz 05,1994,7.0,fr FRA,FWMF,28.0,13.5,6.0,5.0,...,4015,Mixed_Race,134,8,1994/2/25,0,0,22559.0,NaN,85103818
4,Kevin Akpoguma,Hoffenheim,1995,8.0,ng NGA,DF,27.0,21.7,75.0,44.0,...,4747,Mixed_Race,133,11,1995/4/19,4,0,19164.0,NaN,91109389


In [35]:
combined_data.to_csv('data/FB_FBRef_combined.csv')